# Find Tipping Points

## The purpose of this notebook is to use probabilistic programming to find businesses with tipping points. I analyze all of the businesses within the SQL databases and add the results to a new database which is then used to provide the labels for the classifier built in the YelpHelp Notebook.

## Import modules

In [4]:
#Load from the yelphelp package I wrote to get functions for querying our databases,
#processing the reviews to derive features, building the bag of words representation
#and visualizing the results
from yelphelp.queries import query_business_switchpoints, query_business_reviews
from yelphelp.pp_switchpoints import mcmc_changepoint
from yelphelp.data_prep import get_business_class, sample_reviews, scale_features
from yelphelp.nlp_tools import create_nlp_features, get_pos_dist, service_complaint
from yelphelp.viz_tools import make_roc

#SQL dependencies
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

#I/O
import json
import os
import time
import datetime

#Analysis/Data organization
import numpy as np
import pandas as pd
from collections import Counter
from scipy import sparse
from scipy import interpolate
import scipy as sp
import scipy.io
import math
from sklearn.utils import shuffle

#NLP & Feature Analysis
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn import model_selection, preprocessing, ensemble
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.naive_bayes import MultinomialNB

import nltk
from nltk.tokenize import RegexpTokenizer  
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Visualization/Plotting
import matplotlib.pyplot as plt

#Probabilistic Programming
import pymc3 as pm
from pymc3 import Model, DiscreteUniform, Exponential, Poisson, Normal 
from pymc3 import HalfNormal, NUTS, Metropolis, sample, traceplot, find_MAP
from pymc3.math import switch

#Modeling
import xgboost as xgb 
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import log_loss

#Saving results
import pickle

#Visualization
from bokeh.plotting import figure, output_file, show
from wordcloud import WordCloud

# Connect to the SQL databases with the businesses and reviews

In [5]:
#Establish the names for our two databases so we can create them and later query them
reviews_database = 'reviews'
business_database = 'businesses'

username = 'steven' ##This user name must match what's set up in PostgreSQL!

business_con = None
business_con = psycopg2.connect(database = business_database, user = username)
review_con = None
review_con = psycopg2.connect(database = reviews_database, user = username)

#Here I run a query to get the businesses to include in the modeling. This query only has to be run once
#and creates the Pandas dataframe "business_data_from_sql" that I will use to know which reviews to pull
#from the reviews database.
sql_query = """
SELECT * FROM business_data_table WHERE categories LIKE '%Restaurants%' AND longitude < -60 AND review_count > 40;
"""

business_data_from_sql = pd.read_sql_query(sql_query,business_con)

# Run probabilistic programming to find switchpoints

### Note: analyzing a business takes about 15s, so running this on all 13000 businesses would take a very long tim. The way I do this is to analyze in batches of 10 and then add them to the SQL database. So we begin by gettng a list of the businesses that have not yet been analyzed.

## First off, create an SQL database to hold the results of our analysis

In [ ]:
#Create a new SQL database if it doesn't exist
switch_database = 'switchpoints'
switch_engine = create_engine('postgres://%s@localhost/%s'%(username,switch_database))

if not database_exists(switch_engine.url):
    create_database(switch_engine.url)

#Connect to the switch database
switch_con = None
switch_con = psycopg2.connect(database = switch_database, user = username)
    
#Query the switchpoint database--this will give us a list of all the businesses that have already
#been analyzed and we can then skip them and move on to ones we haven't processed yet
switches_already_found = query_business_switchpoints(switch_con)


## Analyze remaining businesses

In [3]:
#Get all of the businesses we haven't already analyzed
remaining = list(filter(lambda x: x not in switches_already_found['business_id'].values,business_data_from_sql['business_id'].values))
print('There are ' + str(len(remaining)) + ' out of ' + str(len(business_data_from_sql)) + ' left to be processed')

#Define the variables we will need
busi_changes = [] #A list of the magnitude of the change after the tipping point
busi_switches = [] #A list of the review # that had a tipping point
busi_ids = [] #A list of the ids of the businesses we've processed 
mcmc_iterations = 5000 #How many iteractions of the Monte Carlo to approximate the distributions

count_businesses_added = 0 #Counter for how many businesses we've processed so far
how_many = 10 #After how many should we add a batch to the database?

for busi_ind in remaining: 
    print(busi_ind)
    
    #Get all of the reviews for this business
    business_reviews = query_business_reviews(busi_ind,review_con)
    #Run the MCMC to find the most likely switch point
    changes, switches, a, b, switch_dist = mcmc_changepoint(business_reviews['date'],business_reviews['user_normed_stars'],mcmc_iter=mcmc_iterations,plot_result=0)
   
    #Add these statistics to our list
    busi_changes.append(changes[0])
    busi_switches.append(switches[0])
    busi_ids.append(business_reviews['business_id'][0])
    count_businesses_added += 1
    
    #If we've completed a batch, then save it to the switch SQL database, clear our variables, and start again
    if np.logical_and( (count_businesses_added % how_many) == 0, count_businesses_added > 0):
        switch_df = pd.DataFrame({'business_id':busi_ids, 'Differential':busi_changes, 'SwitchPoint':busi_switches} )
        print(switch_df)
        switch_df.to_sql('switchpoint_data_table', switch_engine, if_exists='append')
        busi_changes = []
        busi_switches = []
        busi_ids = []
        count_businesses_added = 0

There are 4088 out of 13721 left to be processed
dW79jPJVpzlTKGF-1JAsaw


KeyboardInterrupt: 